In [ ]:
# prompt: 드라이브

from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


## 5차 과제 - 난 어떤 연예인을 닮았을까?
- 주제: 닮은 연예인 찾기
- 데이터: 연예인 이미지 데이터(구글 이미지 크롤링)
- 사용 라이브러리: BeautifulSoup, selenium, PIL, os, tensorflow
- 사용 툴: Teachable Machine
- 참고 자료 및 출처  
  
https://velog.io/@moon_happy/%EC%97%B0%EC%98%88%EC%9D%B8-%EC%82%AC%EC%A7%84-%ED%81%AC%EB%A1%A4%EB%A7%81-%ED%95%98%EA%B8%B0

https://dk-kang.tistory.com/entry/%EC%BC%80%EB%9D%BC%EC%8A%A4-%EB%8B%AE%EC%9D%80%EA%BC%B4-%EC%97%B0%EC%98%88%EC%9D%B8-%EC%B0%BE%EA%B8%B0with-OpenCV-TM#google_vignette

### 1. 이미지 크롤링

1.1 라이브러리 import

In [ ]:
!pip install selenium

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image
import io
import time
import numpy as np
import cv2
import os
from tqdm import tqdm

In [ ]:
!apt-get update

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 49 not upgraded.
Need to get 27.3 MB of archives.
After this operation, 114 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.3 [595 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.ljsu2Nxq30/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.V2oT93Z7YE/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.wcDJQeMltI/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

1.2 크롬드라이버 경로 설정

In [ ]:
# 크롬드라이버 위치 설정
DRIVER_PATH = '/content/drive/MyDrive/Colab Notebooks/chromedriver'

1.3 연예인 리스트 만들기(자유롭게 추가하셔도 됩니다!)

In [ ]:
listm = [
    "이병헌", "김태희", "송혜교", "이정재", "송중기", "전지현", "현빈", "한가인", "박서준", "박보검",
    "김소현", "박신혜", "이종석", "공효진", "공유", "김유정", "수지", "김래원", "한효주", "이민호",
    "유재석", "아이유", "윤아", "차승원", "신민아", "김수현", "정우성", "송지효", "이동욱", "신세경",
    "하정우", "이선균", "김혜수", "이하늬", "장동건", "이명박", "장나라", "탐켄치", "황정음", "고현정",
    "유아인", "손예진", "김래원", "강동원", "나문희", "감스트", "이제훈", "박해일", "공효진", "조승우",
    "정경호", "카리나", "빅맘", "설윤", "마동석", "강호동", "이수지", "탁재훈", "김구라", "정종철",
    "박휘순", "조진세", "윈터", "싸이", "박명수", "조정치", "신지", "문채원", "문재인", "이유영"
]

1.4 연예인 이미지 크롤링 및 폴더 생성

In [ ]:
with tqdm(total=len(listm)) as pbar:
    for i in listm:
        #options = Options()
        service = Service('/usr/bin/chromedriver')
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless')  # 창 안띄우기

        # 검색어 설정
        search_name = i
        # 검색어를 이용한 구글 이미지 검색 url
        url = f'https://www.google.com/search?q={search_name}+얼굴&source=lnms&tbm=isch'
        # 크롬 드라이버 실행
        driver = webdriver.Chrome(service = service, options=chrome_options)
        # url 접속
        driver.get(url)
        # 페이지 로드를 위한 대기 시간
        time.sleep(2)
        # 이미지 로딩을 위한 스크롤 다운
        # 페이지의 초기 높이
        initial_height = driver.execute_script("return document.body.scrollHeight")
        # 페이지 맨 아래로 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        # 페이지의 새로운 높이
        new_height = driver.execute_script("return document.body.scrollHeight")

        # 페이지의 초기 높이와 새로운 높이를 비교하여 스크롤이 되었는지 확인합니다.
        if new_height > initial_height:
            print("Scrolling successful.")
        else:
            print("No scrolling occurred.")
        # 이미지 링크 추출
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        img_tags = soup.find_all('img')
        urls = []
        for img in img_tags:
          try:
              url = img['src']
              if 'http' in url:
                  urls.append(url)
          except:
              pass
        # 이미지 다운로드
        os.makedirs(f'./images{search_name}', exist_ok=True)
        count = 0
        for url in urls:
          try:
              response = requests.get(url, stream=True)
              # 이미지 사이즈 확인
              img = Image.open(io.BytesIO(response.content))
              width, height = img.size
              if width >= 20 and height >= 20:
                  file_name = f'./images{search_name}/{count}.jpg'
                  with open(file_name, 'wb') as out_file:
                      out_file.write(response.content)
                  #print(f'{file_name} saved')
                  count += 1
                  if count == 20:
                      break
          except:
              pass
        # 크롬 드라이버 종료
        driver.quit()

        # tqdm 업데이트
        pbar.update(1)

  0%|          | 0/70 [00:00<?, ?it/s]


WebDriverException: Message: Service /usr/bin/chromedriver unexpectedly exited. Status code was: 1


1.5 얼굴만 검출하기(의상, 전신 사진 등 보정)

In [ ]:
face_cascade = cv2.CascadeClassifier("C:\\Users\\bumbk\\Downloads\\haarcascade_frontalface_alt.xml")

In [ ]:
#경로에 한글이 있어 새로운 imread 함수 설정
def imread_korean(path):
    with open(path, "rb") as fp:
        bytes = bytearray(fp.read())
        numpy_array = np.asarray(bytes, dtype=np.uint8)
    return cv2.imdecode(numpy_array, cv2.IMREAD_UNCHANGED)

In [ ]:
#얼굴만 검출후 연예인_face 파일 생성 및 추가
def save_face(actor) :
    try :
        file_count = len([name for name in os.listdir(f"./images{actor}/") if os.path.isfile(os.path.join(f"./images{actor}/", name))])
        for i in range(file_count) :
            save_img_dir = f"./images{actor}/{i}.jpg"
            new_save_img_dir = f"./images{actor}_face"
            img = imread_korean(f"{save_img_dir}")
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.3,5)

            for (x,y,w,h) in faces:
                actor_face = img[y:y+h, x:x+w]
                try:
                    if not os.path.exists(new_save_img_dir) :
                        os.makedirs(new_save_img_dir)
                except OSError as e:
                    print(e)
                _, im_write_korean = cv2.imencode(".jpg", actor_face)
                im_write_korean.tofile(f"{new_save_img_dir}/{actor}_{i}.jpg")
    except :
        pass

In [ ]:
for actor in listm :
    save_face(actor)

### 2. 가능하다면 여기까지..?

Teachable Machine에서 학습한 모델을 다운받은 후 tensorflow 코드를 받아 파이썬 내에서 실행하는 코드입니다.  
다만 tensorflow 최신 버전(2.16으로 알고 있음)에서는 돌아가지 않는 것으로 확인되며 돌리시려면 버전을 2.12로 낮추고 경로와 파일에 한글이 전혀 없도록 하셔야합니다.

In [ ]:
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("<IMAGE_PATH>").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


TypeError: Error when deserializing class 'DepthwiseConv2D' using config={'name': 'expanded_conv_depthwise', 'trainable': True, 'dtype': 'float32', 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'activity_regularizer': None, 'bias_constraint': None, 'depth_multiplier': 1, 'depthwise_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'depthwise_regularizer': None, 'depthwise_constraint': None}.

Exception encountered: Unrecognized keyword arguments passed to DepthwiseConv2D: {'groups': 1}